In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
import pandas as pd
import tensorflow as tf
#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

In [ ]:
application_df.columns.tolist()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME', ORGANIZATION.
application_df1 = application_df.drop(columns=["EIN", "NAME"])

In [ ]:
# Determine the number of unique values in each column.
application_cat = application_df1.dtypes[application_df1.dtypes == "object"].index.tolist()

application_df1[application_cat].nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df.APPLICATION_TYPE.value_counts()
app_counts

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
app_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_app = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in replace_app:
    application_df1.APPLICATION_TYPE = application_df1.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df1.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df1.CLASSIFICATION.value_counts()
class_counts

In [ ]:
# Visualize the value counts of CLASSIFICATION
class_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df1.CLASSIFICATION = application_df1.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df1.CLASSIFICATION.value_counts()

In [ ]:
import numpy as np
# Add Earned Income column
application_df1["Earned_Income"] = np.where(application_df1["INCOME_AMT"] != "0", "Yes", "No")
application_df1.head()

In [ ]:
!pip install keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
# Generate our categorical variable lists
application_cat = application_df1.dtypes[application_df1.dtypes == "object"].index.tolist()
application_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df1[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
app_df = application_df1.merge(encode_df, left_index=True, right_index=True)
app_df = app_df.drop(application_cat, 1)
app_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = app_df.IS_SUCCESSFUL
X = app_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    number_input_features = len(X_train_scaled[0])

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
!pip install -- upgrade tensorflow

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=5)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
best_hyper = tuner.get_best_hyperparameters()[0]
print(best_hyper.values)

In [ ]:
model = tuner.hypermodel.build(best_hyper)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [ ]:
# Train the model
fit_model = model.fit(X_train_scaled,y_train,epochs=500,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

[For Model 3]()